In [ ]:
#pip install lyricsgenius
import lyricsgenius as lg
#api_key is your TOKEN from Genius Developers. It will look like that.
api_key = "AVD4yMSZU3BsiGEpnohBIbnPj0EweXieJIgCVsaxYxrbxQAqWrN6nCaOQdsH9f3B"
genius = lg.Genius(api_key)
genius = lg.Genius(api_key, skip_non_songs=True, remove_section_headers=True)
#genius.verbose = False
genius.timeout = 120

In [ ]:
import pandas
#data = pandas.read_excel("name_artist_rank.xlsx")
data = pandas.read_excel("end_data.xlsx")
print(data[:5])

songs = data['song'].tolist()
print(songs[0:10])

artists = data['artist'].tolist()
print(artists[0:10])

#peak_rank = data['peak.rank'].tolist()
#print(peak_rank[0:10])

In [ ]:
#remove whatever is after Featuring, &, With, X and x, Genius has problems with that
for i in range (0, len(artists)):
    if "Featuring" in str(artists[i]):
        artists[i] = artists[i].split("Featuring", 1)[0]
        
for i in range (0, len(artists)):
    if "&" in str(artists[i]):
        artists[i] = artists[i].split("&", 1)[0] 
        
for i in range (0, len(artists)):
    if "With" in str(artists[i]):
        artists[i] = artists[i].split("With", 1)[0]

for i in range (0, len(artists)):
    if "X " in str(artists[i]):
        artists[i] = artists[i].split("X ", 1)[0]

for i in range (0, len(artists)):
    if "x" in str(artists[i]):
        artists[i] = artists[i].split("X ", 1)[0]
        
import re        
for i in range (0, len(songs)): #delete anything what's in the brackets in title of the song
    if "(" in songs[i]:
        songs[i] = re.sub("[\(\[].*?[\)\]]", "", songs[i])

In [ ]:
#no_lyrics = pandas.read_excel('you_no_lyrics.xlsx')
#no_lyrics = no_lyrics[0].tolist()

#print(len(no_lyrics))

#data = data.drop(no_lyrics)
#songs = data['song'].tolist()
#print(songs[0:10])

#artists = data['artist'].tolist()
#print(artists[0:10])

In [ ]:
import string

you = []

for i in range (0, len(songs)):
    count = 0
    song = genius.search_song(songs[i], artists[i])
    try:
        lyrics = song.lyrics
        lyrics = str(lyrics)
        lyrics = re.sub("[\(\[].*?[\)\]]", "", lyrics) #deleting everything what is in brackets as those are usually vocalizations or repeated previous word/phrase
        lyrics = lyrics.replace("\n", " ").replace("Embed", "").rstrip("1234567890")
        lyrics = lyrics.translate(str.maketrans('', '', string.punctuation))
        lyrics = lyrics.lower()
        lyrics_list = lyrics.split(" ")
        while '' in lyrics_list:
            lyrics_list.remove('')
        while ' ' in lyrics_list:
            lyrics_list.remove(' ')
        for el in lyrics_list:
            if el == "you" or el == "your" or el == "yours" or el == "youve" or el == "youre" or el == "youll":
                count += 1
        name_of_song = songs[i].lower().split(" ")
        for el in name_of_song:
            if el == "you" or el == "your" or el == "yours":
                count = count - 1
        you.append(count)
        
    except AttributeError: #sometimes the code can't find the lyrics, then the information is added to the list
        you.append("no lyrics found")
        pass
    
print(you)

In [ ]:
import pandas as pd

df = pd.DataFrame(you)
writer = pd.ExcelWriter('you.xlsx', engine='xlsxwriter')
df.to_excel(writer, index=False)
writer.save()
print("Done")